# 7e – Projected Voronoi service areas

Create **Voronoi polygons** (in a planar EPSG:27700 projection) for the acute hospitals; any CDC/CH falling within a polygon is mapped to that hub.  

In [ ]:

import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from scipy.spatial import Voronoi, voronoi_plot_2d
import numpy as np

DATA_DIR = Path('.')
ACUTE_CSV = DATA_DIR / 'NHS_SW_Acute_Hospitals_enriched.csv'
CDC_CSV   = DATA_DIR / 'NHS_SW_Community_Diagnostic_Centres_enriched.csv'
CH_CSV    = DATA_DIR / 'NHS_SW_Community_Hospitals_enriched.csv'

acute = pd.read_csv(ACUTE_CSV)
cdc   = pd.read_csv(CDC_CSV)
ch    = pd.read_csv(CH_CSV)

# Project to British National Grid (EPSG:27700) for planar Voronoi
acute_gdf = gpd.GeoDataFrame(acute,
    geometry=gpd.points_from_xy(acute.longitude, acute.latitude),
    crs='EPSG:4326').to_crs(27700)

coords = np.column_stack([acute_gdf.geometry.x, acute_gdf.geometry.y])
vor = Voronoi(coords)

# Simple plot
fig, ax = plt.subplots(figsize=(8,10))
voronoi_plot_2d(vor, ax=ax, show_points=False, line_colors='grey')
acute_gdf.plot(ax=ax, color='red', markersize=30)
plt.title('Voronoi service areas – acute hospitals')
plt.show()

# Assign each spoke to polygon it falls inside
spokes = pd.concat([cdc, ch], ignore_index=True)
spokes_gdf = gpd.GeoDataFrame(spokes,
    geometry=gpd.points_from_xy(spokes.longitude, spokes.latitude),
    crs='EPSG:4326').to_crs(27700)

import shapely.geometry as sgeom
# Build polygons from voronoi regions
polygons = []
for region_idx in vor.point_region:
    verts = vor.regions[region_idx]
    if -1 in verts or len(verts)==0:  # skip infinite regions
        polygons.append(None)
    else:
        poly = sgeom.Polygon(vor.vertices[verts])
        polygons.append(poly)

acute_gdf['voronoi_poly'] = polygons

def point_to_hub(point):
    for idx, poly in acute_gdf[['Name','voronoi_poly']].iterrows():
        if poly.voronoi_poly and poly.voronoi_poly.contains(point):
            return poly.Name
    return None

spokes_gdf['nearest_acute'] = spokes_gdf.geometry.apply(point_to_hub)
spokes_gdf[['Name', 'nearest_acute']].head()
